In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import dask.dataframe as dd

In [2]:
diag = dd.read_csv('C:\\Users\\yahya\\Documents\\GitHub\\Internship\\CSV_files\\diagnoses_table.csv')
diag = diag.sort_values(by=['subject_id', 'hadm_id'])


In [3]:
def detect_icd_version(code):
    code = str(code).strip().upper()
    return 10 if code and code[0].isalpha() else 9

# Apply to your DataFrame
diag['icd_version'] = diag['icd_code'].apply(detect_icd_version)

c:\Users\yahya\Documents\GitHub\Internship\venv\Lib\site-packages\dask\dataframe\dask_expr\_collection.py:4392: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('icd_code', 'int64'))

  warnings.warn(meta_warning(meta))


In [4]:
diag.head(25)

,subject_id,hadm_id,icd_code,icd_version
0,10000032,29079034,45829,9
1,10000032,29079034,07044,9
2,10000032,29079034,7994,9
3,10000032,29079034,2761,9
4,10000032,29079034,78959,9
5,10000032,29079034,2767,9
6,10000032,29079034,3051,9
7,10000032,29079034,V08,10
8,10000032,29079034,V4986,10
9,10000032,29079034,V462,10


In [5]:
adm = dd.read_csv('C:\\Users\\Yahya\\Documents\\GitHub\\Internship\\CSV_files\\admissions_table.csv')
adm = adm.sort_values(by=['subject_id', 'hadm_id'])

In [6]:
adm.head(25)

,subject_id,hadm_id,admittime,dischtime
0,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00
1,10000117,22927623,2181-11-15 02:05:00,2181-11-15 14:52:00
2,10000117,27988844,2183-09-18 18:10:00,2183-09-21 16:30:00
3,10000635,20642640,2143-12-23 14:55:00,2143-12-24 12:52:00
4,10000635,26134563,2136-06-19 14:24:00,2136-06-20 11:30:00
5,10000690,23280645,2150-09-16 19:48:00,2150-09-24 13:50:00
6,10000690,25860671,2150-11-02 18:02:00,2150-11-12 13:45:00
7,10000690,26146595,2152-01-28 23:40:00,2152-01-30 15:56:00
8,10000690,26504700,2150-07-03 02:46:00,2150-07-07 15:42:00
9,10000764,27897940,2132-10-14 23:31:00,2132-10-19 16:30:00


In [7]:
diag_df = diag.merge(adm, on=['subject_id', 'hadm_id'])

In [8]:
diag_df.head(25)

,subject_id,hadm_id,icd_code,icd_version,admittime,dischtime
0,10000032,29079034,45829,9,2180-07-23 12:35:00,2180-07-25 17:55:00
1,10000032,29079034,07044,9,2180-07-23 12:35:00,2180-07-25 17:55:00
2,10000032,29079034,7994,9,2180-07-23 12:35:00,2180-07-25 17:55:00
3,10000032,29079034,2761,9,2180-07-23 12:35:00,2180-07-25 17:55:00
4,10000032,29079034,78959,9,2180-07-23 12:35:00,2180-07-25 17:55:00
5,10000032,29079034,2767,9,2180-07-23 12:35:00,2180-07-25 17:55:00
6,10000032,29079034,3051,9,2180-07-23 12:35:00,2180-07-25 17:55:00
7,10000032,29079034,V08,10,2180-07-23 12:35:00,2180-07-25 17:55:00
8,10000032,29079034,V4986,10,2180-07-23 12:35:00,2180-07-25 17:55:00
9,10000032,29079034,V462,10,2180-07-23 12:35:00,2180-07-25 17:55:00


In [9]:
lab = dd.read_csv('C:\\Users\\yahya\\Documents\\GitHub\\Internship\\CSV_files\\labevents_combined.csv')

In [10]:
diag_df['admittime'] = dd.to_datetime(diag_df['admittime'])
diag_df['dischtime'] = dd.to_datetime(diag_df['dischtime'])

In [11]:
diag_d = dd.read_csv('C:\\Users\\yahya\\Documents\\GitHub\\Internship\\CSV_files\\icd_details.csv')


In [12]:
print(diag_df['icd_code'].dtype)
print(diag_d['icd_code'].dtype)

string
int64


In [13]:
diag_df['icd_code'] = diag_df['icd_code'].astype(str)
diag_d['icd_code'] = diag_d['icd_code'].astype(str)

In [14]:
diag_df = diag_d.merge(diag_df, on = 'icd_code')

In [ ]:
diagnosis_events = diag_df.copy()
diagnosis_events['Start_Date'] = diagnosis_events['admittime'].dt.strftime('%Y,%m,%d')
diagnosis_events['End_Date'] = diagnosis_events['dischtime'].dt.strftime('%Y,%m,%d')
diagnosis_events['headline'] = "Diagnosis: " + diagnosis_events['icd_code'].astype(str)
diagnosis_events['text'] = diagnosis_events['long_title']
diagnosis_events['group'] = "Diagnosis"
diagnosis_events['Media'] = ""
diagnosis_events['Media_Credit'] = ""
diagnosis_events['Media_Caption'] = ""
diagnosis_events['Type'] = ""

diagnosis_timeline = diagnosis_events[['Start_Date', 'End_Date', 'headline', 'text', 'Media', 'Media_Credit', 'Media_Caption', 'group', 'Type']]

In [ ]:
lab['charttime'] = dd.to_datetime(lab['charttime'])

In [ ]:
lab_events = lab.copy()
lab_events['Start_Date'] = lab_events['charttime'].dt.strftime('%Y,%m,%d')
lab_events['End_Date'] = ""
lab_events['headline'] = "Lab: " + lab_events['label'].astype(str)
lab_events['text'] = "Value: " + lab_events['valuenum'].astype(str) + lab_events['valueuom'].astype(str)
lab_events['group'] = "Lab"
lab_events['Media'] = ""
lab_events['Media_Credit'] = ""
lab_events['Media_Caption'] = ""
lab_events['Type'] = ""

lab_timeline = lab_events[['Start_Date', 'End_Date', 'headline', 'text', 'Media', 'Media_Credit', 'Media_Caption', 'group', 'Type']]


In [ ]:
timeline_df = dd.concat([diagnosis_timeline, lab_timeline], ignore_index=True)
timeline_df = timeline_df.sort_values(by='Start_Date')

In [ ]:
import streamlit as st
import dask.dataframe as dd
import pandas as pd
import tempfile
import os

# Load minimal data just to populate dropdown
@st.cache_data
def get_unique_subject_ids():
    return diag_df["subject_id"].drop_duplicates().compute().sort_values().tolist()

subject_ids = get_unique_subject_ids()
selected_id = st.selectbox("Select a subject_id", subject_ids)

if selected_id:
    st.write(f"Generating timeline for subject_id: {selected_id}")
    diagnosis_events = diag_df.copy()
    diagnosis_events['Start_Date'] = diagnosis_events['admittime'].dt.strftime('%Y,%m,%d')
    diagnosis_events['End_Date'] = diagnosis_events['dischtime'].dt.strftime('%Y,%m,%d')
    diagnosis_events['headline'] = "Diagnosis: " + diagnosis_events['icd_code'].astype(str)
    diagnosis_events['text'] = diagnosis_events['long_title']
    diagnosis_events['group'] = "Diagnosis"
    diagnosis_events['Media'] = ""
    diagnosis_events['Media_Credit'] = ""
    diagnosis_events['Media_Caption'] = ""
    diagnosis_events['Type'] = ""
    
    diagnosis_timeline = diagnosis_events[['Start_Date', 'End_Date', 'headline', 'text', 'Media', 'Media_Credit', 'Media_Caption', 'group', 'Type']]

    lab_events = lab.copy()
    lab_events['Start_Date'] = lab_events['charttime'].dt.strftime('%Y,%m,%d')
    lab_events['End_Date'] = ""
    lab_events['headline'] = "Lab: " + lab_events['label'].astype(str)
    lab_events['text'] = "Value: " + lab_events['valuenum'].astype(str) + lab_events['valueuom'].astype(str)
    lab_events['group'] = "Lab"
    lab_events['Media'] = ""
    lab_events['Media_Credit'] = ""
    lab_events['Media_Caption'] = ""
    lab_events['Type'] = ""
    
    lab_timeline = lab_events[['Start_Date', 'End_Date', 'headline', 'text', 'Media', 'Media_Credit', 'Media_Caption', 'group', 'Type']]

    timeline_df = dd.concat([diagnosis_timeline, lab_timeline], ignore_index=True)
    timeline_df = timeline_df.sort_values(by='Start_Date')

    with tempfile.NamedTemporaryFile(delete=False, suffix=".csv", mode='w', newline='', encoding='utf-8') as tmp:
        timeline_df.to_csv(tmp.name, index=False)
        st.success("Timeline generated!")
        st.download_button("Download Timeline CSV", tmp.read(), file_name=f"timeline_subject_{selected_id}.csv", mime="text/csv")
    
    os.remove(tmp.name)

In [33]:
unique_codes = diag['icd_code'].unique().compute()
sorted_codes = unique_codes.sort_values()
print(sorted_codes.tolist())

['0020', '0030', '0031', '00329', '0038', '0039', '0040', '0041', '0048', '0049', '0050', '00581', '00589', '0059', '0062', '0068', '0071', '0074', '0078', '00800', '00804', '0083', '00841', '00843', '00845', '00846', '00847', '00849', '0085', '00861', '00862', '00863', '00867', '00869', '0088', '0090', '0091', '0092', '0093', '01120', '01123', '01124', '01126', '01140', '01163', '01180', '01186', '01190', '01192', '01193', '01194', '01195', '01196', '01203', '01205', '01206', '01215', '01300', '01325', '01400', '01402', '01405', '01485', '01500', '01501', '01505', '01522', '01690', '01720', '01724', '01730', '01796', '01804', '01890', '01894', '01895', '01896', '0218', '0239', '0270', '0272', '0310', '0311', '0312', '0318', '0319', '03284', '03285', '03289', '0329', '0339', '0340', '035', '0360', '0361', '0362', '0380', '03810', '03811', '03812', '03819', '0382', '0383', '03840', '03841', '03842', '03843', '03844', '03849', '0388', '0389', '0390', '0391', '0392', '0393', '0398', '0400

In [52]:
icd9 = diag.copy()
icd9['icd_code'] = dd.to_numeric(icd9['icd_code'], errors='coerce')
diag_cml_9 = icd9[(icd9['icd_code'].astype(float) >= 20500) & (icd9['icd_code'].astype(float) <= 20591)]
diag_cml_9 = diag_cml_9.reset_index(drop=True)

diag_cml_9.compute().head(25)

,subject_id,hadm_id,icd_code,icd_version
0,10035631,20385771,20500,9
1,10035631,22732862,20500,9
2,10035631,24912093,20501,9
3,10035631,27496788,20500,9
4,10035631,29462354,20500,9
5,10035631,29654498,20500,9
6,10070928,20163692,20500,9
7,10070928,20297201,20500,9
8,10070928,20721218,20500,9
9,10070928,22119100,20500,9


In [53]:
n_rows = diag_cml_9.shape[0].compute()
n_cols = diag_cml_9.shape[1]
print(f"Shape: ({n_rows}, {n_cols})")

Shape: (1865, 4)


In [50]:
icd10 = diag[diag['icd_code'].str.match(r'^[A-Z]')]

diag_cml_10 = icd10[icd10['icd_code'].str.startswith('C92')]
diag_cml_10 = diag_cml_10.reset_index(drop=True)

diag_cml_10.compute().head(25)

,subject_id,hadm_id,icd_code,icd_version
0,10010231,20687038,C92Z0,10
1,10010231,21586397,C9200,10
2,10010231,23083980,C9200,10
3,10010231,23835132,C92Z0,10
4,10010231,24995642,C92Z0,10
5,10010231,25499227,C9200,10
6,10010231,27998273,C92Z1,10
7,10010231,28743978,C9200,10
8,10010231,29368887,C9200,10
9,10026643,27759132,C9210,10


In [51]:
n_rows = diag_cml_10.shape[0].compute()
n_cols = diag_cml_10.shape[1]
print(f"Shape: ({n_rows}, {n_cols})")

Shape: (2134, 4)


In [54]:
diag_cml = dd.concat([diag_cml_9, diag_cml_10])
n_rows = diag_cml.shape[0].compute()
n_cols = diag_cml.shape[1]
print(f"Shape: ({n_rows}, {n_cols})")

Shape: (3999, 4)
